In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import os
import imageio
from PIL import Image

from tqdm import tqdm
from visdom import Visdom
from tensorboardX import SummaryWriter

from torch.utils.data import DataLoader
import Reader

from dotmap import DotMap

config = DotMap({
    "training": True,
    "batch": 32,
    "workers": 0,
    "epoch": 400
})

In [2]:
# 1x1 convolution
def conv_one(in_channel, out_channel):
    return nn.Sequential(
        nn.BatchNorm3d(in_channel),
        nn.ReLU(),
        nn.Conv3d(in_channel, out_channel, 1, bias=False)
    )

In [3]:
# residual block
class Residual(nn.Module):
    def __init__(self, in_channel, out_channel=None):
        super(Residual, self).__init__()
        self.in_channel = in_channel
        self.out_channel = out_channel
        
        if out_channel is None:
            out_channel = in_channel
        
        self.conv_block = nn.Sequential(
            conv_one(in_channel, out_channel // 2),
            nn.BatchNorm3d(out_channel // 2),
            nn.ReLU(),
            nn.Conv3d(out_channel // 2, out_channel // 2, 3, padding=1, bias=False),
            conv_one(out_channel // 2, out_channel)
        )
        
        self.skip_layer = None
        if in_channel != out_channel:
            self.skip_layer = nn.Conv3d(in_channel, out_channel, 1)
        
        self.out_relu = nn.ReLU(inplace=True)
        
    def forward(self, x):
        skip = x
        
        out = self.conv_block(x)
        
        if self.skip_layer is not None:
            skip = self.skip_layer(x)
        
        out += skip
        
        return out

In [4]:
# hourglass
class Hourglass(nn.Module):
    def __init__(self, size, channels):
        super(Hourglass, self).__init__()
        self.size = size
        self.channels = channels
        
        self.skip_layers = nn.ModuleList([Residual(self.channels) for _ in range(self.size)])
        self.low_layers = nn.ModuleList([Residual(self.channels) for _ in range(self.size)])
        self.mid = Residual(self.channels)
        self.up_layers = nn.ModuleList([Residual(self.channels) for _ in range(self.size)])
        self.max_pool = nn.MaxPool3d((1, 2, 2), stride=(1, 2, 2))
        self.upsample = nn.Upsample(scale_factor=(1, 2, 2), mode='trilinear', align_corners=False)
        # nearest not working with tuple(1, 2, 2). so bilinear

    def forward(self, x):
        inner = x

        skip_outputs = list()
        for skip, low in zip(self.skip_layers, self.low_layers):
            s = skip(inner)
            skip_outputs.append(s)
            inner = self.max_pool(inner)
            inner = low(inner)
        
        out = self.mid(inner)

        for skip, up in zip(reversed(skip_outputs), reversed(self.up_layers)):
            out = skip + self.upsample(up(out))

        return out

In [5]:
# model
class Model(nn.Module):
    def __init__(self, features, internal_size=4):
        super(Model, self).__init__()
        self.features = features
        self.internal_size = internal_size
        
        self.init_conv = nn.Sequential(
            nn.Conv3d(1, 8, 7, stride=(1, 2, 2), padding=3, bias=False),
            nn.BatchNorm3d(8),
            nn.ReLU(),
            Residual(8, 16),
            nn.MaxPool3d((1, 2, 2), stride=(1, 2, 2)),
            Residual(16, 32),
            Residual(32, self.features)
        )
        
        self.hourglass = Hourglass(self.internal_size, self.features)
        
        self.out_conv = nn.Sequential(
            Residual(self.features, self.features),
            conv_one(self.features, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        init = self.init_conv(x)
           
        hg = self.hourglass(init)
        
        out = self.out_conv(hg)

        return out

In [ ]:
loader = DataLoader(
    Reader.Liver3Layer(
        augment = True
    ),
    config.batch,
    shuffle=(config.training),
    pin_memory=True,
    num_workers=config.workers
)

In [ ]:
# training
model = Model(64)
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=0.0001)#lr=0.00025)
criterion = nn.BCELoss(torch.ones(config.batch, dtype=torch.float64))
# cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

training = True

visual = Visdom()
writer = SummaryWriter()

#Batch, Channel, Depth, Height, Width
dummy_input = torch.autograd.Variable(
    torch.rand(config.batch, 1, 3, 256, 256)
)
dummy_input = dummy_input.to(device)
writer.add_graph(model, (dummy_input, ))

In [ ]:
training = True

counter = 0

torch.set_num_threads(4)
for epoch in range(1, config.epoch):
    with tqdm(total=len(loader), unit=' iter', unit_scale=False) as progress:
        progress.set_description('Epoch %d' % epoch)
        
        with torch.set_grad_enabled(training):
            for images, labels in loader:
                optimizer.zero_grad()

                images = images.float()
                images = images.to(device)

                labels = labels.float()
                labels = labels.to(device)
                
                output = model(images)
                
                if training:
                    out_image = output.cpu().data
                    label_image = labels.cpu().data
                    
                    weight = []
                    for images in label_image.numpy().squeeze():
                        weight_batch = []
                        for image in images:
                            value = np.ceil(image)
                            if np.sum(image) > 0:
                                value *= (np.sum(1-image) / np.sum(image))-1
                            value += 1
                            weight_batch.append(value)
                        weight.append(weight_batch)
                    
                    weight = np.array(weight)
                    
                    criterion.weight = torch.tensor(weight).to(device)
                    
                    loss = criterion(output.squeeze(), labels.squeeze())
                    loss = torch.mean(loss)
                    loss.backward()
                    
                    optimizer.step()
                    progress.set_postfix(loss=float(loss.item()))
                    
                    out_image = out_image.view(-1, 64, 64)
                    out_image = out_image.numpy()
                    out_image = out_image.squeeze()
                    out_depth = len(out_image)
                    out_image = out_image.repeat(3, axis=0)
                    out_image = np.reshape(out_image, (out_depth, 3, 64, 64))
                    
                    label_image = label_image.view(-1, 64, 64)
                    label_image = label_image.numpy()
                    label_image = label_image.squeeze()
                    label_depth = len(label_image)
                    label_image = label_image.repeat(3, axis=0)
                    label_image = np.reshape(label_image, (label_depth, 3, 64,64))
                    
                    visual.images(
                        tensor=out_image, nrow=12,
                        win='output',
                        opts=dict(title='output')
                    )
                    visual.images(
                        tensor=label_image, nrow=12,
                        win='label',
                        opts=dict(title='label')
                    )
                    
                    writer.add_scalar('data/loss', float(loss.item()), counter)

                    progress.update(1)
                    
                    counter += 1
                    
                else:
                    raise Exception('There\'s no Validation code available.')

In [ ]:
# save model
save_epoch = config.epoch
save_epoch = 0
torch.save({
    'epoch': save_epoch,
    'state': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'save/3_'+str(save_epoch)+'.save')

In [6]:
#load model
model = Model(64)
pretrained_model = torch.load('save/3_100_cb.save')
model.load_state_dict(pretrained_model['state'])

model = model.eval()
# cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

training = False

visual = Visdom()

In [11]:
loader = DataLoader(
    Reader.LiverData(
        augment = False,
        paths = ['3Dircadb/']
    ),
    1,
    shuffle=False,
    pin_memory=True,
    num_workers=config.workers
)

In [13]:
# test a CT
training = False

torch.set_num_threads(4)

num_valid_data = 0
meanVOE = 0
meanDICE = 0
meanRVD = 0
DICE_list = []

with tqdm(total=len(loader), unit=' iter', unit_scale=False) as progress:
    progress.set_description('Testing')

    with torch.set_grad_enabled(training):
        for image, label in loader:
            
            image = image.float()
            image = image.squeeze()
            
            output = []
            
            for index in range(0, len(image), 3):
                final = 0
                if index+3 > len(image):
                    final = index+3 - len(image)
                    index = len(image) - 3
                    
                input_cell = image[index:index+3]
                input_cell = input_cell.unsqueeze(0).unsqueeze(0)
                input_cell = input_cell.to(device)
                out_cell = model(input_cell)
                out_cell = out_cell.squeeze()
                if final is 0:
                    feed = out_cell.cpu().data.numpy()
                    for layer in feed:
                        output.append(layer)
                else:
                    feed = out_cell.narrow(0, final, 3-final)
                    feed = feed.cpu().data.numpy()
                    for layer in feed:
                        output.append(layer)
            
            label = label.float()
            label = label.to(device)

            if training:
                raise Exception('There\'s no Training code available.')
            else:
                input_image = image.cpu().data
                input_image = input_image.view(-1, 256, 256)
                input_image = input_image.numpy()
                input_image = input_image.squeeze()
                input_image = [Image.fromarray(layer, 'F') for layer in input_image]
                input_image = [layer.resize((64, 64), Image.ANTIALIAS) for layer in input_image]
                input_image = [np.array(layer) for layer in input_image]
                input_image = np.asarray(input_image)
                input_image[input_image < 0] = 0
                input_image *= 255
                input_depth = len(input_image)
                input_image = input_image.repeat(3, axis=0)
                input_image = np.reshape(input_image, (input_depth, 3, 64, 64))
                
                out_image = np.asarray(output)
                out_image = out_image.reshape(-1, 64, 64)
                out_image = out_image.squeeze()
                out_depth = len(out_image)
                out_image = out_image.repeat(3, axis=0)
                out_image = np.reshape(out_image, (out_depth, 3, 64, 64))
                
                label_image = label.cpu().data
                label_image = label_image.view(-1, 64, 64)
                label_image = label_image.numpy()
                label_image = label_image.squeeze()
                label_depth = len(label_image)
                label_image = label_image.repeat(3, axis=0)
                label_image = np.reshape(label_image, (label_depth, 3, 64,64))
                
#                 for out_layer, label_layer in zip(out_image, label_image):
#                     out_layer[0][:] = 0 #R
#                     out_layer[1] = label_layer[1] #G
#                     out_layer[2][:] = 0 #B

                visual.images(
                    tensor=input_image, nrow=9,
                    win='input',
                    opts=dict(title='input')
                )
                visual.images(
                    tensor=out_image, nrow=9,
                    win='output',
                    opts=dict(title='output')
                )
                visual.images(
                    tensor=label_image, nrow=9,
                    win='label',
                    opts=dict(title='label')
                )

                #threshold
                out_image = np.ceil(out_image - 0.6)

                a = np.sum(label_image)
                b = np.sum(out_image)
                aub = np.sum((label_image + out_image) / 2)
                anb = np.sum(label_image * out_image)

                if a > 1:
                    #Volume Overlap Error
                    VOE = 1 - (anb / aub)

                    #DICE score
                    DICE = (2 * anb) / (a + b)

                    #Relative Volume Difference
                    RVD = (b - a) / a

                    meanVOE += VOE
                    meanDICE += DICE
                    meanRVD += RVD

                    num_valid_data += 1
                    
                    DICE_list.append(DICE)
                
                progress.set_postfix(loss=float(DICE))

                progress.update(1)

meanVOE /= num_valid_data
print('VOE:  ', meanVOE)
meanDICE /= num_valid_data
print('DICE: ', meanDICE)
meanRVD /= num_valid_data
print('RVD:  ', meanRVD)
print(DICE)

Testing: 100%|██████████| 20/20 [00:15<00:00,  1.20 iter/s, loss=0.585]

VOE:   0.36785492088113514
DICE:  0.6321451005228319
RVD:   -0.04113172287387507
0.5846636499350428


In [ ]:
num = 0
for value in DICE_list:
    print(value)
    if value > 0.1:
        num += 1
print(sum(DICE_list)/num)

In [ ]:
criterion.weight

In [ ]:
# test model
training = False

torch.set_num_threads(4)

num_valid_data = 0
meanVOE = 0
meanDICE = 0
meanRVD = 0

with tqdm(total=len(loader), unit=' iter', unit_scale=False) as progress:
    progress.set_description('Testing')

    with torch.set_grad_enabled(training):
        for image, label in loader:
            
            image = image.float()
            image = image.to(device)
            
            label = label.float()
            label = label.to(device)
            
            output = model(image)
            
            if training:
                raise Exception('There\'s no Training code available.')
            else:
                input_image = image.cpu().data
                input_image = input_image.view(-1, 256, 256)
                input_image = input_image.numpy()
                input_image = input_image.squeeze()
                input_image = [Image.fromarray(layer, 'F') for layer in input_image]
                input_image = [layer.resize((64, 64), Image.ANTIALIAS) for layer in input_image]
                input_image = [np.array(layer) for layer in input_image]
                input_image = np.asarray(input_image)
                input_image[input_image < 0] = 0
                input_image *= 255
                input_depth = len(input_image)
                input_image = input_image.repeat(3, axis=0)
                input_image = np.reshape(input_image, (input_depth, 3, 64, 64))
                
                out_image = output.cpu().data
                out_image = out_image.view(-1, 64, 64)
                out_image = out_image.numpy()
                out_image = out_image.squeeze()
                out_depth = len(out_image)
                out_image = out_image.repeat(3, axis=0)
                out_image = np.reshape(out_image, (out_depth, 3, 64, 64))
                
                label_image = label.cpu().data
                label_image = label_image.view(-1, 64, 64)
                label_image = label_image.numpy()
                label_image = label_image.squeeze()
                label_depth = len(label_image)
                label_image = label_image.repeat(3, axis=0)
                label_image = np.reshape(label_image, (label_depth, 3, 64,64))
                
#                 for out_layer, label_layer in zip(out_image, label_image):
#                     out_layer[0][:] = 0 #R
#                     out_layer[1] = label_layer[1] #G
#                     out_layer[2][:] = 0 #B

                visual.images(
                    tensor=input_image, nrow=12,
                    win='input',
                    opts=dict(title='input')
                )
                visual.images(
                    tensor=out_image, nrow=12,
                    win='output',
                    opts=dict(title='output')
                )
                visual.images(
                    tensor=label_image, nrow=12,
                    win='label',
                    opts=dict(title='label')
                )

                #threshold
                out_image = np.ceil(out_image - 0.5)

                for label_layer, out_layer in zip(label_image, out_image):
                    #BASIC values
                    a = np.sum(label_layer)
                    b = np.sum(out_layer)
                    aub = np.sum((label_layer + out_layer) / 2)
                    anb = np.sum(label_layer * out_layer)

                    if a > 0:
                        #Volume Overlap Error
                        VOE = 1 - (anb / aub)

                        #DICE score
                        DICE = (2 * anb) / (a + b)

                        #Relative Volume Difference
                        RVD = (b - a) / a

                        meanVOE += VOE
                        meanDICE += DICE
                        meanRVD += RVD

                        num_valid_data += 1

                progress.update(1)

meanVOE /= num_valid_data
print('VOE:  ', meanVOE)
meanDICE /= num_valid_data
print('DICE: ', meanDICE)
meanRVD /= num_valid_data
print('RVD:  ', meanRVD)

In [ ]:
loader = DataLoader(
    Reader.Liver3Layer(
        augment = False
    ),
    config.batch,
    shuffle=False,
    pin_memory=True,
    num_workers=config.workers
)

In [ ]:
# test one part
training = False

torch.set_num_threads(4)

num_valid_data = 0
meanVOE = 0
meanDICE = 0
meanRVD = 0

with tqdm(total=len(loader), unit=' iter', unit_scale=False) as progress:
    progress.set_description('Testing')

    with torch.set_grad_enabled(training):
        image, label = Reader.Liver3Layer(augment=False).__getitem__(5504)
        
        image = image.unsqueeze(0)
        image = image.float()
        image = image.to(device)

        label = label.unsqueeze(0)
        label = label.float()
        label = label.to(device)

        output = model(image)

        if training:
            raise Exception('There\'s no Training code available.')
        else:
            input_image = image.cpu().data
            input_image = input_image.view(-1, 256, 256)
            input_image = input_image.numpy()
            input_image = input_image.squeeze()
            input_image = [Image.fromarray(layer, 'F') for layer in input_image]
            input_image = [layer.resize((64, 64), Image.ANTIALIAS) for layer in input_image]
            input_image = [np.array(layer) for layer in input_image]
            input_image = np.asarray(input_image)
            input_image[input_image < 0] = 0
            input_image *= 255
            input_depth = len(input_image)
            input_image = input_image.repeat(3, axis=0)
            input_image = np.reshape(input_image, (input_depth, 3, 64, 64))

            out_image = output.cpu().data
            out_image = out_image.view(-1, 64, 64)
            out_image = out_image.numpy()
            out_image = out_image.squeeze()
            out_depth = len(out_image)
            out_image = out_image.repeat(3, axis=0)
            out_image = np.reshape(out_image, (out_depth, 3, 64, 64))

            label_image = label.cpu().data
            label_image = label_image.view(-1, 64, 64)
            label_image = label_image.numpy()
            label_image = label_image.squeeze()
            label_depth = len(label_image)
            label_image = label_image.repeat(3, axis=0)
            label_image = np.reshape(label_image, (label_depth, 3, 64,64))

#                 for out_layer, label_layer in zip(out_image, label_image):
#                     out_layer[0][:] = 0 #R
#                     out_layer[1] = label_layer[1] #G
#                     out_layer[2][:] = 0 #B

            visual.images(
                tensor=input_image, nrow=12,
                win='input',
                opts=dict(title='input')
            )
            visual.images(
                tensor=out_image, nrow=12,
                win='output',
                opts=dict(title='output')
            )
            visual.images(
                tensor=label_image, nrow=12,
                win='label',
                opts=dict(title='label')
            )

            #threshold
            out_image = np.ceil(out_image - 0.5)

            for label_layer, out_layer in zip(label_image, out_image):
                    #BASIC values
                    a = np.sum(label_layer)
                    b = np.sum(out_layer)
                    aub = np.sum((label_layer + out_layer) / 2)
                    anb = np.sum(label_layer * out_layer)

                    if a > 0:
                        #Volume Overlap Error
                        VOE = 1 - (anb / aub)

                        #DICE score
                        DICE = (2 * anb) / (a + b)

                        #Relative Volume Difference
                        RVD = (b - a) / a

                        meanVOE += VOE
                        meanDICE += DICE
                        meanRVD += RVD

                        num_valid_data += 1

            progress.update(1)

meanVOE /= num_valid_data
print('VOE:  ', meanVOE)
meanDICE /= num_valid_data
print('DICE: ', meanDICE)
meanRVD /= num_valid_data
print('RVD:  ', meanRVD)